<a href="https://colab.research.google.com/github/olgOk/QCircuit/blob/master/tutorials/Deutsch_Algorithm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Deutsch Algorithm

by Olga Okrut

Install frameworks, and import libraries

In [1]:
!pip install tensornetwork jax jaxlib colorama qcircuit

     |████████████████████████████████| 225kB 4.7MB/s 
  Created wheel for qcircuit: filename=qcircuit-1.0.1-cp36-none-any.whl size=5988 sha256=506a90f6dbd06114774149fbfe798d9bb83491e2361e49d43b4cf708f70d415c
  Stored in directory: /root/.cache/pip/wheels/a3/dd/39/d3dece77c172e493c47e48a0b28382d116ff2daad674a7270b
Successfully built qcircuit
  Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1


In [0]:
from qcircuit import QCircuit as qc

Now, after we have learned how quantum gates work and how to build a quantum circuit, we will jump to the first quantum algorithm. We begin with a very simple quantum algorithm - Deutsch algorithm, named after its inventor David Deutsch, which serves as an excellent proof of the supremacy of quantum computers and algorithms over classical.


The problem Deutsch algorithm tackles can now be stated as follows. Given a black box *Uf* implementing
some unknown binary function *f* that maps {0, 1} into {0, 1}.
We have to clasify *f* as “constant” or “balanced” function. 



Here, constant means function always outputs the same bit, i.e. f(0) = f(1) = 1 or f(0) = f(1) = 0:
![picture](https://drive.google.com/uc?id=1CytjIW8-GO1KZfybNH21XtzTFWf9iTFX)



Balanced means function outputs different bits on different inputs, i.e. f(0) != f(1):
![picture](https://drive.google.com/uc?id=1WST_QiyQ9HSR_K98HAsmIkZB8fssPOWg)

The circuit for Deutsch’s algoritm is given below. The steps for the Deutsch algorithm:

1.   Prepare two qubits, one in state `1|0> + 0|1>` and the other in state `0|0> + 1|1>` (apply *X* gate on the second qubit).

2.  Apply the Hadamard gate (*H*) on both qubits to bring them to superposition.

3. The output after the Hadamard transformation will be send through the gate *Uf*. The values of the *Uf* matrix depends on the *f(x)* function. That means that the state vector after the gate *Uf* depends on the function, e.g. constant or balanced function.

4. The output from the *Uf* transormation is send to the gates Hadarard again. It will collapse the state vector from the superposition to one of the possible state depending on the function *f(x)*. 

5. The output from the Hadamard transformation will be a two qubit register. If all four possible function values are tested, it is revealed that the final output will be either `(0, 0), (0, 1), (1, 0), or (1, 1)` with probability of 1. The output value will depend on *f(x)*. The two qubits are entangled in the end, so only one of their values can be measured. This prevents us from known exactly which *f(x)* is being used. However, the first qubit in the pair will always be 1 if the function *f(x)* is **balanced**. If *f(x)* is **constant**, the algorithm outputs 0.

![picture](https://drive.google.com/uc?id=1rGNVTM3xl6AUQ6__k7LY0eSbot4W_tZU)



Now, let's create the quantum circuit above. We will use built-in method 
```
Uf(function)
```
which translates a classcal binary function *f(x)* into a unitary matrix *U*, and applies it to the circuit. As a parameter, it takes a function that needs to be tested for being balanced or constant. I will use a set of predefined functions to show the validity of the algorithm.

In [4]:
# define binary functions. Some of them are constant, other balanced
def f1(x):
    return x

def f2(x):
    return 1

def f3(x):
    return 0

def f4(x):
    return not x

def f5(x):
    return x ** 2

def f6(x):
    return not (x ** 3)

def f7(x):
  return (x % 3 == 2)

def f8(x):
  return not (x % 3 == 2)

# check if the function constant
def is_const(func):
  deutsch = qc.QCircuit(2)
  deutsch.X(0)
  deutsch.H(1)
  deutsch.H(0)
  deutsch.Uf(func)
  deutsch.H(0)
  deutsch.H(1)

  # get output state vector
  # decide if a function constanta or balanced
  output_state = deutsch.get_state_vector()
  if abs(output_state[0]) == 1.+0.j or abs(output_state[1]) == 1.+0.j:
    return True
  else:
    return False

functions = [f1, f2, f3, f4, f5, f6, f7, f8]

for func in range(len(functions)):
  print('function f{} is {}'.format(func+1, 'constant' if is_const(functions[func]) else 'balansed'))

/usr/local/lib/python3.6/dist-packages/jax/lib/xla_bridge.py:123: UserWarning: No GPU/TPU found, falling back to CPU.
  warnings.warn('No GPU/TPU found, falling back to CPU.')


function f1 is balansed
function f2 is constant
function f3 is constant
function f4 is balansed
function f5 is balansed
function f6 is balansed
function f7 is constant
function f8 is constant
